# Tek Yönlü Anova Analizi

In [87]:
import pandas as pd
import numpy as np
from scipy import stats
import scikit_posthocs as sp

In [88]:
country_name = ["France", "Greece",	"Italy", "Portugal", "Spain", "Türkiye"]

In [89]:
df_2 = pd.read_excel("OECD VERİLERİ ELİF BAĞCI TEZ V0_python.xlsx", sheet_name=10, header=1 )
df_2.head()

,Unnamed: 0,France,Greece,Italy,Portugal,Spain,Türkiye,t,interruption,interrupted t
0,2000,16.899,19.871,21.158,21.566,21.958,27.806000,1,0,0
1,2001,17.333,19.075,21.920,22.192,21.949,35.512848,2,0,0
2,2002,17.188,19.949,21.865,22.255,22.650,37.713939,3,0,0
3,2003,17.559,24.572,21.375,22.043,21.418,36.040784,4,0,0
4,2004,17.630,24.716,20.996,22.096,20.677,34.124759,5,0,0


### 1. Adım: Varsayımların sağlandığının kontrolü

    Her gruptaki değerler kendi içinde normal dağılım göstermelidir.
    Grup varyansları eşit olmalıdır.

In [90]:
# Normallik Varsayımı

# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.

In [91]:
for country in country_name:    
    test_stat, pvalue = stats.shapiro(df_2[f"{country}"])
    print("############################################")
    print(f"{country}'nin Shapiro-Wilk Testi Sonuçları")
    print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}', end="\n\n")
    if pvalue<0.05:
        print("p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Seri normal dağılmamıştır.", end="\n\n")
    elif pvalue>0.05:
        print("p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani Seri normal dağılmıştır.", end="\n\n")

############################################
France'nin Shapiro-Wilk Testi Sonuçları
Test Stat = 0.9067, p-value = 0.0472

p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Seri normal dağılmamıştır.

############################################
Greece'nin Shapiro-Wilk Testi Sonuçları
Test Stat = 0.8502, p-value = 0.0043

p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Seri normal dağılmamıştır.

############################################
Italy'nin Shapiro-Wilk Testi Sonuçları
Test Stat = 0.9318, p-value = 0.1497

p-value 0.05'ten büyüktür. H0 hipotezi reddedilememiştir. Yani Seri normal dağılmıştır.

############################################
Portugal'nin Shapiro-Wilk Testi Sonuçları
Test Stat = 0.8293, p-value = 0.0019

p-value 0.05'ten küçüktür. H0 hipotezi reddedilmiştir. Seri normal dağılmamıştır.

############################################
Spain'nin Shapiro-Wilk Testi Sonuçları
Test Stat = 0.8728, p-value = 0.0108

p-value 0.05'ten küçüktür. H0 hipotezi reddedil

    Normallik varsayımı sağlanamıyor. Yine de varyans homojenliğini kontrol edelim.

In [92]:
# Varyans homojenliği

# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir

In [93]:
test_stat, pvalue = stats.levene(df_2["France"],
                                 df_2["Greece"],
                                 df_2["Italy"],
                                 df_2["Portugal"],
                                 df_2["Spain"],
                                 df_2["Türkiye"],)

print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}')

# Aşağıda görüldüğü üzere varyans homojenliği varsayımı sağlanmamaktadır.

Test Stat = 8.4057, p-value = 0.0


    Burada non-parametrik anova testi uygulanması gerekmektedir. Bunun için biz kruskal testini uygulauacağız.

In [94]:
# Kruskal testinin hipotezleri

# H0: Grupların ortalamaları arasındaki fark anlamlı değildir.
# H1: En az iki grup ortalaması arasındaki fark istatistiksel olarak anlamlıdır.

In [95]:
stats.kruskal(df_2["France"],
              df_2["Greece"],
              df_2["Italy"],
              df_2["Portugal"],
              df_2["Spain"],
              df_2["Türkiye"])

# p value 0.05'ten küçük olduğu için H0 hipotezini reddederiz. 
# Yani, en az iki grup ortalaması arasındaki fark istatistiksel olarak anlamlıdır.

KruskalResult(statistic=89.28874962058313, pvalue=9.477999445566358e-18)

In [105]:
dunns_test = sp.posthoc_dunn([df_2["France"],
                              df_2["Greece"],
                              df_2["Italy"],
                              df_2["Portugal"],
                              df_2["Spain"],
                              df_2["Türkiye"]], p_adjust = 'bonferroni')

dunns_test.columns = country_name
dunns_test.insert(0, column="", value= country_name)
dunns_test.set_index("", inplace=True)
dunns_test

,France,Greece,Italy,Portugal,Spain,Türkiye
,,,,,,
France,1.000000e+00,9.872941e-11,0.021636,0.091501,9.449327e-01,1.429731e-13
Greece,9.872941e-11,1.000000e+00,0.003493,0.000558,8.283011e-06,1.000000e+00
Italy,2.163576e-02,3.492796e-03,1.000000,1.000000,1.000000e+00,7.697846e-05
Portugal,9.150080e-02,5.582929e-04,1.000000,1.000000,1.000000e+00,8.466719e-06
Spain,9.449327e-01,8.283011e-06,1.000000,1.000000,1.000000e+00,5.930097e-08
Türkiye,1.429731e-13,1.000000e+00,0.000077,0.000008,5.930097e-08,1.000000e+00


In [106]:
dunns_test_analy = dunns_test.applymap(lambda x:"Reject" if x<0.05 else "Fail to reject")
dunns_test_analy

,France,Greece,Italy,Portugal,Spain,Türkiye
,,,,,,
France,Fail to reject,Reject,Reject,Fail to reject,Fail to reject,Reject
Greece,Reject,Fail to reject,Reject,Reject,Reject,Fail to reject
Italy,Reject,Reject,Fail to reject,Fail to reject,Fail to reject,Reject
Portugal,Fail to reject,Reject,Fail to reject,Fail to reject,Fail to reject,Reject
Spain,Fail to reject,Reject,Fail to reject,Fail to reject,Fail to reject,Reject
Türkiye,Reject,Fail to reject,Reject,Reject,Reject,Fail to reject


In [97]:
data_list = []

for country in country_name:
    for i in df_2[f"{country}"]:
        data_list.append(i)

country_list = []

for country in country_name:
    for i in df_2[f"{country}"]:
        country_list.append(f"{country}")

df_3 = pd.DataFrame({"data": data_list, "country_list": country_list })
df_3.head()

,data,country_list
0,16.899,France
1,17.333,France
2,17.188,France
3,17.559,France
4,17.630,France


In [98]:
# Tukey testinin hipotezleri;

# Ho: Gruplar arası ortalamalara göre istatistiksel olarak anlamlı fark yoktur.
# Hı: Gruplar arası ortalamalara göre istatistiksel olarak anlamlı fark vardır.

In [99]:
from statsmodels.stats.multicomp import MultiComparison
comparison = MultiComparison(df_3['data'], df_3['country_list'])    
tukey = comparison.tukeyhsd(0.05)
print(tukey.summary())

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1   group2  meandiff p-adj   lower    upper  reject
---------------------------------------------------------
  France   Greece   10.922    0.0   8.2563 13.5877   True
  France    Italy   3.5548 0.0025   0.8891  6.2205   True
  France Portugal   3.2662 0.0072   0.6005  5.9319   True
  France    Spain   2.2105 0.1639  -0.4552  4.8762  False
  France  Türkiye  14.1178    0.0  11.4521 16.7835   True
  Greece    Italy  -7.3672    0.0 -10.0329 -4.7015   True
  Greece Portugal  -7.6558    0.0 -10.3215 -4.9901   True
  Greece    Spain  -8.7115    0.0 -11.3772 -6.0458   True
  Greece  Türkiye   3.1957 0.0092     0.53  5.8614   True
   Italy Portugal  -0.2886 0.9996  -2.9543  2.3771  False
   Italy    Spain  -1.3443 0.6898    -4.01  1.3214  False
   Italy  Türkiye   10.563    0.0   7.8973 13.2287   True
Portugal    Spain  -1.0557 0.8605  -3.7214    1.61  False
Portugal  Türkiye  10.8515    0.0   8.1858 13.5172   True
   Spain  Türk

#### Kaynakça

    https://istatistikturkiye.com/post-hoc-testlerinin-dogru-kullanimi/
    https://www.statology.org/tukey-test-python/
    https://www.statology.org/dunns-test-python/
